In [76]:
from typing import List
import math
Vector = List[float]
def dot(v: Vector, w: Vector) -> float:
    assert len(v) == len(w)
    return sum(v_i * w_i for v_i, w_i in zip(v, w))

def step_function(x:float)->float:
    return 1 if x >= 0 else 0
def sigmoid(t:float)->float:
    return 1/(1+math.exp(-t))

def neuron_output(weights:Vector,inputs:Vector)->float:
    return sigmoid(dot(weights,inputs))

def perceptron_output(weights:Vector,inputs:Vector)->float:
    return step_function(dot(weights,inputs))


def feed_forward(neural_network: List[List[Vector]],
                 input_vector: Vector) -> List[Vector]:
    ouputs: List[Vector] = []
    
    for layer in neural_network:
        input_with_bias = input_vector + [1]
        output = [neuron_output(neuron, input_with_bias)
                  for neuron in layer]
        ouputs.append(output)
        input_vector = output
    return ouputs

xor_network = [
    [[20.0, 20.0, -30.0], [20.0, 20.0, -10.0]],
    [[-60.0, 60.0, -30.0]]
]
assert feed_forward(xor_network, [0, 0])[-1][0] < 0.01
assert feed_forward(xor_network, [1, 0])[-1][0] > 0.99
assert feed_forward(xor_network, [0, 1])[-1][0] > 0.99
assert feed_forward(xor_network, [1, 1])[-1][0] < 0.01


In [77]:
#back propagation
from typing import List
Vector = List[float]
def dot(v: Vector, w: Vector) -> float:
    assert len(v) == len(w)
    return sum(v_i * w_i for v_i, w_i in zip(v, w))
def sqerror_gradient(network:List[List[Vector]],
                     input_vector: Vector,
                     target_vector: Vector) -> List[List[Vector]]:
    hidden_outputs, outputs = feed_forward(network, input_vector)
    output_deltas=[output*(1-output)*(output-target)
                   for output, target in zip(outputs, target_vector)]
    
    output_grads=[[output_deltas[i] *hidden_output
                   for hidden_output in hidden_outputs + [1]]
                  for i ,output_neuron in enumerate(network[-1])]
    
    hidden_deltas=[hidden_output*(1-hidden_output)*
                   dot(output_deltas,[n[i] for n in network[-1]])
                   for i, hidden_output in enumerate(hidden_outputs)]
    
    hidden_grads=[[hidden_deltas[i]*input for input in input_vector + [1]]
                    for i, hidden_neuron in enumerate(network[0])]
    return [hidden_grads, output_grads]
#Test the back propagation
xor_network = [
    [[20.0, 20.0, -30.0], [20.0, 20.0, -10.0]],
    [[-60.0, 60.0, -30.0]]
]
sqerror_gradient(xor_network, [1, 0], [1])

[[[2.4142328917571065e-29, 0.0, 2.4142328917571065e-29],
  [-2.4142328917547655e-29, -0.0, -2.4142328917547655e-29]],
 [[-4.023904162934704e-31, -8.863238740646161e-27, -8.863641131062455e-27]]]

In [78]:
def gradient_step(vectors:List[Vector],gradients:List[Vector],step_size:float)->List[Vector]:
    step=[step_size*gradient for gradient in gradients]
    return [vector+step for vector, step in zip(vectors,step)]

In [79]:
def get_error(network:List[List[Vector]],
                xs:List[Vector],
                ys:List[Vector])->float:
        error=0.0
        assert len(xs)==len(ys)
        for x,y in zip(xs,ys):
            predicted=feed_forward(network,x)[-1][0]
            error+= (predicted-y[0])**2
        return error

In [80]:
get_error(xor_network,[[0,0],[0,1],[1,0],[1,1]],[[0],[1],[1],[0]])

3.533597059709085e-26

In [94]:
import random
import tqdm
#Training
learning_rate=0.1
xs=[[0,0],[0,1],[1,0],[1,1]]
ys=[[0],[1],[1],[0]]
network=[[ [random.random() for _ in range(2+1)],
              [random.random() for _ in range(2+1)]],
             [[random.random() for _ in range(2+1)]]]

In [95]:
get_error(network,xs,ys)

1.2771063157711633

In [96]:

with tqdm.trange(10000)as t:
    for _ in range(10000):
        for x,y in zip(xs,ys):
            gradients=sqerror_gradient(network,x,y)
            network=[[gradient_step(neuron,grad,-learning_rate)
                    for neuron,grad in zip(layer,layer_grad)]
                    for layer,layer_grad in zip(network,gradients)]
        t.update(1)
        t.set_description(f"error: {get_error(network,xs,ys):.2f}")


        

error: 0.01: 100%|██████████| 10000/10000 [00:30<00:00, 328.87it/s]


In [97]:
get_error(network,xs,ys)

0.011837266332907225

In [98]:
print(network)

[[[3.716659600083313, 3.7151985311512936, -5.688597240171938], [5.841141354190779, 5.838514807537462, -2.437853846000429]], [[-8.176106026697012, 7.528019906900595, -3.388102741245321]]]


In [100]:
#Testing 
print(feed_forward(network,[0,0])[-1][0])
print(feed_forward(network,[0,1])[-1][0])  
print(feed_forward(network,[1,0])[-1][0])
print(feed_forward(network,[1,1])[-1][0])

0.05673562347588216
0.94781024480601
0.9477774038262772
0.05627935023947783


In [101]:
network

[[[3.716659600083313, 3.7151985311512936, -5.688597240171938],
  [5.841141354190779, 5.838514807537462, -2.437853846000429]],
 [[-8.176106026697012, 7.528019906900595, -3.388102741245321]]]